# 3D Triangulation for hdf5 files for a session
Triangulate the 3D locations of animals using [anipose](https://anipose.readthedocs.io/en/latest/index.html)<br>
We start with the tracked 2D locations of the animals, stored in separate hdf5 files for each camera view, along with pre-computed calibration configuration.<br>

# Imports

In [32]:
# Generic imports
from os import listdir
from os.path import join, split
import numpy as np 
import matplotlib.pyplot as plt 
import h5py
import re
from scipy.io import savemat
import importlib
from IPython.utils import io

In [9]:
# Custom libraries 
spec = importlib.util.find_spec('aniposelib')
if spec is None:
    #print(package_name +" is not installed")
    !python -m pip install --user aniposelib
from aniposelib.cameras import CameraGroup
from preprocessing import get_2d_poses
from triangulation import TriangulationMethod, get_3d_poses

### Init variables

In [31]:
# Init path with session data
session_filepath = 'C:\\Users\\shantanu.ray\\Downloads'
_, session_name = split(session_filepath)
save_filepath = 'C:\\Users\\shantanu.ray\\Downloads'
# Camera name in the h5 file (must be the same for a given session)
camera_ids = ['cam1','cam2','cam3']
# Pre-computed calibration file (must be the same for a given session)
calibration_file = 'C:\\Users\\shantanu.ray\\projects\\sleap\\addons\\calibration.toml'

### Init triangulation parameters

In [20]:
# Choose triangulation method
triangulate = TriangulationMethod.calibrated_dtl
refine_calibration = True
show_progress = False

### Find hdf5 files

In [11]:
h5_regex = re.compile('.*cam.*.h5')
file_list = list(filter(h5_regex.match, listdir(session_filepath)))
print('Read files from session path')
print(file_list)

['CFL12_04072022_CNO_27_14-39-25_cam1.analysis.h5', 'CFL12_04072022_CNO_27_14-39-25_cam2.analysis.h5', 'CFL12_04072022_CNO_27_14-39-25_cam3.analysis.h5', 'CFL12_04072022_saline_27_14-39-24_cam1.analysis.h5', 'CFL12_04072022_saline_27_14-39-24_cam2.analysis.h5', 'CFL12_04072022_saline_27_14-39-24_cam3.analysis.h5']


### Find groups of marker data for a given trial for all 3 cameras

Use cam1 as an indicator of the trial

Assumption: All files for a given session have a similar name except cam1 being replaced by cam2,3

In [23]:
cam1_regex = re.compile(f'.*{camera_ids[0]}.*.h5')
cam1_file_list = list(filter(cam1_regex.match, file_list))

# Loading Data for Trial

In [37]:
for cam1_file in cam1_file_list:
    trial_name = cam1_file.replace('_' + camera_ids[0], '')
    print(f'Processing {trial_name}')
    # Get files for camera group (cam1,2,3) for a given trial
    cam_group = [cam1_file.replace(camera_ids[0], cam_idx) for cam_idx in camera_ids]
    """Get 2D data."""
    # The get_2D_poses function returns 2D tracks for a single file at a time,
    # so we append all the tracks to a list and then stack the 2D tracks on top of each other. 
    pose2d = [get_2d_poses(join(session_filepath, file)) for file in cam_group]
    pose2d = np.stack(pose2d, axis=0)[:, :]

    n_cams, n_frames, n_nodes, _, n_tracks = pose2d.shape
    print('2D Data shape', pose2d.shape) # (n_cams, n_sampled_frames, n_keypoints, 2, # tracks)
    """Get 3D data from triangulation."""
    """Aniposelib gives us the option to triangulate with the direct linear transformation (DLT) or with RANSAC,
    which adds an outlier rejection subroutine to the DLT. <br>
    In addition to these 2 triangulation methods, we can further refine the 3D points via direct optimization
    of the reprojection error.
    def get_3d_poses(
                    poses_2d: list,
                    camera_mats: list = [],
                    calibration_filepath: str = None,
                    triangulate: TriangulationMethod = TriangulationMethod.simple,
                    refine_calibration: bool = False,
                    show_progress: bool = False
                    ) -> np.ndarray
    Args:
        poses_2d: A length # cameras list of pose matrices for a single animal. Each pose matrix is of 
        shape (# frames, # nodes, 2, # tracks).
        
        camera_mats: A length # cameras list of camera matrices. Each camera matrix is a (3,4) ndarray. 
        Note that the camera matrices and pose_2d matrices have to be ordered in a corresponding fashion.
        or
        calibration_filepath: Filepath to calibration.toml

        triangulate: Triangulation method
            - simple: No other options are required
            - calibrated_dtl: refine_calibration, show_progress can be passed
            - calibrated_ransac: refine_calibration, show_progress can be passed

        refine_calibration: bool = False, Use CameraGroup.optim refinement

        show_progress: bool = False, Show progress of calibration
        
    Returns:
        poses_3d: A (# frames, # nodes, 3, # tracks) that corresponds to the triangulated 3D points in the world frame. 
    """
    with io.capture_output() as captured:
        pose3d = get_3d_poses(poses_2d=pose2d,
                             calibration_filepath=calibration_file,
                             triangulate=triangulate,
                             refine_calibration=refine_calibration,
                             show_progress=show_progress)
    print('3D Data shape', pose3d.shape)
    mat_dict = {'session': session_name,
                'trial': trial_name.replace('.h5', ''),
                'pose3d': pose3d}
    mat_file = join(save_filepath, trial_name.replace('.h5', '.mat'))
        
    print(f'Saving pose 3d to {mat_file}')
    savemat(mat_file, mat_dict)

Processing CFL12_04072022_CNO_27_14-39-25.analysis.h5
2D Data shape (3, 1600, 11, 2, 1)
3D Data shape (1600, 11, 3, 1)
Saving pose 3d to C:\Users\shantanu.ray\Downloads\CFL12_04072022_CNO_27_14-39-25.analysis.mat
Processing CFL12_04072022_saline_27_14-39-24.analysis.h5
2D Data shape (3, 1600, 11, 2, 1)
3D Data shape (1600, 11, 3, 1)
Saving pose 3d to C:\Users\shantanu.ray\Downloads\CFL12_04072022_saline_27_14-39-24.analysis.mat
